In [24]:

%pip install beautifulsoup4
%pip install lxml
%pip install cloudscraper
%pip install selenium
%pip install nltk

Note: you may need to restart the kernel to use updated packages.


Could not find platform independent libraries <prefix>


Note: you may need to restart the kernel to use updated packages.


Could not find platform independent libraries <prefix>


Note: you may need to restart the kernel to use updated packages.


Could not find platform independent libraries <prefix>


Note: you may need to restart the kernel to use updated packages.


Could not find platform independent libraries <prefix>


Note: you may need to restart the kernel to use updated packages.


Could not find platform independent libraries <prefix>


## 1. Clean data

In [25]:
import re
import os

def clean_wikipedia_text(text):
    # Remove citations like [1], [2][3], [citation needed]
    text = re.sub(r'\[\d+(?:,\d+)*\]|\[citation needed\]', '', text)
    
    # Correct incorrectly escaped HTML entities like `&lt;` and `&gt;`
    text = re.sub(r'&lt;', '<', text)
    text = re.sub(r'&gt;', '>', text)
    
    # Remove URLs
    text = re.sub(r'https?://\S+', '', text)
    
    # Remove parenthetical information
    text = re.sub(r'\([^)]*\)-', '', text)
    
    # Remove special formatting (e.g., ''italic'', """bold""" or similar)
    text = re.sub(r"''+|\"\"+|'''", '', text)
    
    # Remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Remove lines starting with special characters (-, *, digits, or #)
    text = '\n'.join(
        line for line in text.splitlines()
        if not re.match(r'^(\s*[-*#]|\s*\d+\.)', line.strip())
    )
    
    return text


# Ensure the output directory exists
os.makedirs('data_clean', exist_ok=True)

# Load text, clean, and save the output
input_file = 'data.txt'
output_file = 'cleaned_wiki_text.txt'

try:
    with open(input_file, 'r', encoding='utf-8') as file:
        raw_text = file.read()
    
    cleaned_text = clean_wikipedia_text(raw_text)
    
    with open(output_file, 'w', encoding='utf-8') as file:
        file.write(cleaned_text)
    
    print(f"Cleaned text saved to {output_file}.")
except FileNotFoundError:
    print(f"Error: Input file '{input_file}' not found.")
except Exception as e:
    print(f"An error occurred: {e}")


Cleaned text saved to cleaned_wiki_text.txt.


In [26]:
with open('cleaned_wiki_text.txt', 'r', encoding='utf-8') as file:
    cleaned_text = file.read()
    
print(len(cleaned_text.strip(" ")))

149839


## 2. Split text to three part
- 70% for train
- 10% for validation
- 20% for test

In [27]:
import random
import re
import os

# Load text from file
with open('cleaned_wiki_text.txt', 'r', encoding='utf-8') as file:
    text = file.read().lower()
    
text = re.sub(r'\n+', '', text)
text = re.sub(r'\s+', ' ', text)
text = re.sub(r'\[\]', '', text)

# Split text into sentences
sentences = text.split('.')
# print(len(sentences))

# Shuffle sentences
random.shuffle(sentences)

# Split into training, validation, and testing sets
training_sentences = sentences[:int(0.8 * len(sentences))]
validation_sentences = sentences[int(0.8 * len(sentences)):int(0.9 * len(sentences))]
testing_sentences = sentences[int(0.9 * len(sentences)):]

os.makedirs('data', exist_ok=True)
# Write to files
with open('training_data.txt', 'w', encoding='utf-8') as file:
    file.write(''.join(training_sentences))

with open('validation_data.txt', 'w', encoding='utf-8') as file:
    file.write(''.join(validation_sentences))

with open('sting_data.txt', 'w', encoding='utf-8') as file:
    file.write(''.join(testing_sentences))

print("Training, validation, and testing sets saved to 'training_sentences.txt', 'validation_sentences.txt', and 'testing_sentences.txt' respectively.")

Training, validation, and testing sets saved to 'training_sentences.txt', 'validation_sentences.txt', and 'testing_sentences.txt' respectively.


## 3. Create model 1

In [31]:
from nltk.util import ngrams
from collections import defaultdict
import os
import csv

# Count n-grams
def count_ngrams(tokens, n):
    ngram_counts = defaultdict(int)
    for text in tokens:
        for ngram in ngrams(text, n, pad_left=True, pad_right=True):
            ngram_counts[ngram] += 1
    return ngram_counts

# Backoff probability computation
def backoff_probability(ngram, ngram_counts, lower_ngram_counts):
    if ngram_counts[ngram] > 0:
        return ngram_counts[ngram] / lower_ngram_counts[ngram[:-1]]
    elif len(ngram) > 1:
        return backoff_probability(ngram[1:], lower_ngram_counts, unigram_counts)
    else:
        return 0.0


# Save n-gram counts to a file
def save_ngram_counts(ngram_counts, output_file):
    with open(output_file, 'w', encoding='utf-8', newline='') as f:
        writer = csv.writer(f)
        # Write the header
        writer.writerow(['word', 'count'])
        
        # Write the n-gram counts
        for ngram, count in ngram_counts.items():
            ngram_str = ' '.join(str(token) for token in ngram if token is not None)
            writer.writerow([ngram_str, count])
# def save_ngram_counts(ngram_counts, output_file):
#     with open(output_file, 'w', encoding='utf-8') as f:
#         for ngram, count in ngram_counts.items():
#             ngram_str = ' '.join(str(token) for token in ngram if token is not None)
#             f.write(f"{ngram_str}\t{count}\n")

if __name__ == "__main__":
    # Ensure the tokenized data directory exists
    tokenized_data_dir = "./"
    if not os.path.exists(tokenized_data_dir):
        raise FileNotFoundError(f"Directory '{tokenized_data_dir}' not found. Run the tokenization script first.")

    # Load tokenized training data
    train_tokens = []
    with open(os.path.join(tokenized_data_dir, "training_data.txt"), "r", encoding='utf-8') as f:
        for line in f:
            train_tokens.append(line.strip().split())

    # Count n-grams
    unigram_counts = count_ngrams(train_tokens, 1)
    bigram_counts = count_ngrams(train_tokens, 2)
    trigram_counts = count_ngrams(train_tokens, 3)
    fourgram_counts = count_ngrams(train_tokens, 4)

    # Save n-gram counts to files
    output_dir = "LM1_ngram_counts"
    os.makedirs(output_dir, exist_ok=True)
    save_ngram_counts(unigram_counts, os.path.join(output_dir, "unigram_counts.csv"))
    save_ngram_counts(bigram_counts, os.path.join(output_dir, "bigram_counts.csv"))
    save_ngram_counts(trigram_counts, os.path.join(output_dir, "trigram_counts.csv"))
    save_ngram_counts(fourgram_counts, os.path.join(output_dir, "fourgram_counts.csv"))

    print("N-gram counts saved to 'LM1_ngram_counts' directory.")

N-gram counts saved to 'LM1_ngram_counts' directory.
